# Лаббораторная работа 6

In [ ]:
import os
import sys
import json
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 3 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))

In [15]:
from pyspark import SparkContext, SparkConf

conf = SparkConf()
conf.set("spark.app.name", "Alexander Dorofeyev. Spark app for lab06") 

sc = SparkContext(conf=conf)

In [16]:
# Проверяем, что все работает
sc

<SparkContext master=yarn appName=Alexander Dorofeyev. Spark app for lab05>

In [17]:
# лишнаяя строка, лучше удалить
#sc.stop()

### 1. Загрузка данных

In [18]:
# данные по оценкам
user_row_data = sc.textFile("/labs/lab06data/ml-100k/u.data")

In [21]:
# добавить пробелы полсле знака равно для читаемости
user_row_data.takeSample(withReplacement=False, num=10, seed=7007)

['652\t395\t3\t882567383',
 '597\t235\t4\t875340062',
 '361\t657\t5\t879441253',
 '826\t399\t4\t885690790',
 '551\t17\t5\t892784942',
 '264\t655\t4\t886123530',
 '63\t333\t4\t875746917',
 '933\t238\t2\t874853819',
 '16\t95\t5\t877728417',
 '313\t22\t3\t891014870']

In [19]:
# данные по фильмам
movie_raw_data = sc.textFile("/labs/lab06data/ml-100k/u.item")

In [22]:
#добавить пробелы полсле знака равно для читаемости
movie_raw_data.takeSample(withReplacement=False, num=10, seed=7007)

['1044|Paper, The (1994)|01-Jan-1994||http://us.imdb.com/M/title-exact?Paper,%20The%20(1994)|0|0|0|0|0|1|0|0|1|0|0|0|0|0|0|0|0|0|0',
 '933|Funeral, The (1996)|25-Oct-1996||http://us.imdb.com/M/title-exact?Funeral,%20The%20(1996)|0|0|0|0|0|0|0|0|1|0|0|0|0|0|0|0|0|0|0',
 '616|Night of the Living Dead (1968)|01-Jan-1968||http://us.imdb.com/M/title-exact?Night%20of%20the%20Living%20Dead%20(1968)|0|0|0|0|0|0|0|0|0|0|0|1|0|0|0|1|0|0|0',
 '1446|Bye Bye, Love (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Bye%20Bye,%20Love%20(1995)|0|0|0|0|0|1|0|0|0|0|0|0|0|0|0|0|0|0|0',
 '658|Pump Up the Volume (1990)|01-Jan-1990||http://us.imdb.com/M/title-exact?Pump%20Up%20the%20Volume%20(1990)|0|0|0|0|0|0|0|0|1|0|0|0|0|0|0|0|0|0|0',
 '54|Outbreak (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Outbreak%20(1995)|0|1|0|0|0|0|0|0|1|0|0|0|0|0|0|0|1|0|0',
 '522|Down by Law (1986)|01-Jan-1986||http://us.imdb.com/M/title-exact?Down%20by%20Law%20(1986)|0|0|0|0|0|1|0|0|1|0|0|0|0|0|0|0|0|0|0',
 '732|Dave 

### 2. Обработка данных на выход

In [28]:
# нет комментария, не понятно откуда константа
film_id = '168'

In [33]:
# добавить пробелы полсле знака равно для читаемости
# расчет индивидуальных характеристик "моего фильма" в разрезе оценок 1,2,3,4,5
my_film_data = user_row_data.map(lambda x: x.split("\t")).filter(lambda x: x[1]==film_id)\
.map(lambda x: (x[2], 1)).reduceByKey(lambda a,b : a+b).sortByKey().map(lambda x: x[1]).collect()

In [35]:
my_film_data

[12, 6, 56, 117, 125]

In [34]:
# статистика оценок по всем фильмам в разрезе оценок 1,2,3,4,5
all_film_data = user_row_data.map(lambda x: x.split("\t"))\
.map(lambda x: (x[2], 1)).reduceByKey(lambda a,b : a+b).sortByKey().map(lambda x: x[1]).collect()

In [36]:
all_film_data

[6110, 11370, 27145, 34174, 21201]

In [37]:
# добавить пробелы полсле двоеточия для читаемости
final_result = {"hist_film":my_film_data, "hist_all":all_film_data}

In [38]:
final_result

{'hist_film': [12, 6, 56, 117, 125],
 'hist_all': [6110, 11370, 27145, 34174, 21201]}

In [41]:
# сохраняем в json
with open("/data/home/alexander.dorofeyev/lab06.json", 'w') as file:
    json.dump(final_result, file)

In [42]:
sc.stop()